In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
from datetime import datetime
%matplotlib inline

In [2]:
# Gather Data
start = '2018-01-02'
end = '2019-04-01'
FB = web.DataReader('FB', 'iex',start, end)
AMZN = web.DataReader('AMZN', 'iex',start, end)
AAPL = web.DataReader('AAPL', 'iex',start, end)
NFLX = web.DataReader('NFLX', 'iex',start, end)
GOOGL = web.DataReader('GOOGL', 'iex',start, end)

In [3]:
Cash = FB.copy()
Cash['close'] = 1
Cash.head()

,open,high,low,close,volume
date,,,,,
2018-01-02,177.68,181.58,177.5500,1,18151903
2018-01-03,181.88,184.78,181.3300,1,16886563
2018-01-04,184.90,186.21,184.0996,1,13880896
2018-01-05,185.59,186.90,184.9300,1,13574535
2018-01-08,187.20,188.90,186.3300,1,17994726


In [4]:
# Get time series history of stocks
stocks = [FB, AMZN, AAPL, NFLX, GOOGL, Cash]
stock_columns = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL', 'Cash']
all_pos_vals = []

for stock in stocks:
    all_pos_vals.append((stock)['close'])

portfolio_val = pd.concat(all_pos_vals, axis=1)
portfolio_val.columns = stock_columns
portfolio_val.head()

,FB,AMZN,AAPL,NFLX,GOOGL,Cash
date,,,,,,
2018-01-02,181.42,1189.01,168.9872,201.07,1073.21,1
2018-01-03,184.67,1204.20,168.9578,205.05,1091.52,1
2018-01-04,184.33,1209.59,169.7426,205.63,1095.76,1
2018-01-05,186.85,1229.14,171.6751,209.99,1110.29,1
2018-01-08,188.28,1246.87,171.0375,212.05,1114.21,1


In [5]:
# Simulate a starting portfolio
start_value = 1000000
equal_weight = 0.2
tickers = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL','Cash']
d = {'Target':[0.2, 0.2, 0.2, 0.2, 0.2, 0],
    'Upper': 0.25,
    'Lower': 0.15}
start_port = pd.DataFrame(index=tickers,data=d)
start_port['Upper']['Cash'] = 0
start_port['Lower']['Cash'] = 0
start_port

,Target,Upper,Lower
FB,0.2,0.25,0.15
AMZN,0.2,0.25,0.15
AAPL,0.2,0.25,0.15
NFLX,0.2,0.25,0.15
GOOGL,0.2,0.25,0.15
Cash,0.0,0.00,0.00


In [6]:
tickers

['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL', 'Cash']

In [7]:
# Set starting share amount
current_shares = [100,100,100,100,100,0]
start_port['Current Shares'] = current_shares
start_port

,Target,Upper,Lower,Current Shares
FB,0.2,0.25,0.15,100
AMZN,0.2,0.25,0.15,100
AAPL,0.2,0.25,0.15,100
NFLX,0.2,0.25,0.15,100
GOOGL,0.2,0.25,0.15,100
Cash,0.0,0.00,0.00,0


## Calculate the Weights for the time period start (2018-01-02)

In [8]:
current_port = start_port.copy()
current_port

,Target,Upper,Lower,Current Shares
FB,0.2,0.25,0.15,100
AMZN,0.2,0.25,0.15,100
AAPL,0.2,0.25,0.15,100
NFLX,0.2,0.25,0.15,100
GOOGL,0.2,0.25,0.15,100
Cash,0.0,0.00,0.00,0


In [9]:
# Calculate starting price
start_allocation = [100,100,100,100,100,0]
today = '2018-01-02'
price_data_start = pd.DataFrame(portfolio_val.loc[start]).transpose()
price_data_start

,FB,AMZN,AAPL,NFLX,GOOGL,Cash
2018-01-02,181.42,1189.01,168.9872,201.07,1073.21,1.0


In [10]:
# Calculate equity position values
equity_positions = (price_data_start * start_allocation).transpose()
equity_positions

,2018-01-02
FB,18142.00
AMZN,118901.00
AAPL,16898.72
NFLX,20107.00
GOOGL,107321.00
Cash,0.00


In [11]:
# calculate today's portfolio value
port_val_today = equity_positions.sum().sum()
port_val_today

281369.71999999997

In [12]:
# Calculate current weights
weights = equity_positions/port_val_today
weights

,2018-01-02
FB,0.064477
AMZN,0.422579
AAPL,0.060059
NFLX,0.071461
GOOGL,0.381423
Cash,0.000000


In [13]:
# Starting price data
price_data = price_data_start.transpose()
price_data

,2018-01-02
FB,181.4200
AMZN,1189.0100
AAPL,168.9872
NFLX,201.0700
GOOGL,1073.2100
Cash,1.0000


In [14]:
current_port = pd.concat([current_port, price_data, equity_positions, weights], axis=1)
current_port.columns = ['Target', 'Upper', 'Lower', 'Current Share', 'Close', 'MV', 'Current Weight']
current_port

,Target,Upper,Lower,Current Share,Close,MV,Current Weight
FB,0.2,0.25,0.15,100,181.4200,18142.00,0.064477
AMZN,0.2,0.25,0.15,100,1189.0100,118901.00,0.422579
AAPL,0.2,0.25,0.15,100,168.9872,16898.72,0.060059
NFLX,0.2,0.25,0.15,100,201.0700,20107.00,0.071461
GOOGL,0.2,0.25,0.15,100,1073.2100,107321.00,0.381423
Cash,0.0,0.00,0.00,0,1.0000,0.00,0.000000


# Calculate weights on rebalance day

In [15]:
# grab date for today
rebalance_day = '2018-01-03'
rebalance_day

'2018-01-03'

In [16]:
# get rebalance prices
price_at_rebalance = pd.DataFrame(portfolio_val.loc[rebalance_day]).transpose()
price_at_rebalance

,FB,AMZN,AAPL,NFLX,GOOGL,Cash
2018-01-03,184.67,1204.2,168.9578,205.05,1091.52,1.0


In [17]:
# Get equity value for each stock
current_alloc = current_port['Current Share']
equity_positions_rebalance = (price_at_rebalance * current_alloc).transpose()
equity_positions_rebalance

,2018-01-03
FB,18467.00
AMZN,120420.00
AAPL,16895.78
NFLX,20505.00
GOOGL,109152.00
Cash,0.00


In [18]:
# Get portfolio value at rebalance
port_val_rebalance = equity_positions_rebalance.sum().sum()
port_val_rebalance

285439.78000000003

In [19]:
# Get weights at rebalance 
weights_rebalance = equity_positions_rebalance / port_val_rebalance
weights_rebalance

,2018-01-03
FB,0.064697
AMZN,0.421875
AAPL,0.059192
NFLX,0.071837
GOOGL,0.382399
Cash,0.000000


In [20]:
price_at_rebalance=price_at_rebalance.transpose()

In [21]:
current_port = start_port.copy()

current_port = pd.concat([current_port, price_at_rebalance,  equity_positions_rebalance, weights_rebalance], axis=1)
current_port.columns = ['Target', 'Upper', 'Lower', 'Current Share', 'Close', 'MV', 'Current Weight']
current_port

,Target,Upper,Lower,Current Share,Close,MV,Current Weight
FB,0.2,0.25,0.15,100,184.6700,18467.00,0.064697
AMZN,0.2,0.25,0.15,100,1204.2000,120420.00,0.421875
AAPL,0.2,0.25,0.15,100,168.9578,16895.78,0.059192
NFLX,0.2,0.25,0.15,100,205.0500,20505.00,0.071837
GOOGL,0.2,0.25,0.15,100,1091.5200,109152.00,0.382399
Cash,0.0,0.00,0.00,0,1.0000,0.00,0.000000


## Get the result of the current and target weights

In [22]:
# Add conditionals for overweighting and underweighting
def calculate_weights(row):
    current_weight = row[6]
    upper = row[1]
    target = row[0]
    lower = row[2]
    
    if current_weight > upper:
        return('Very Overweight')
    elif current_weight <= upper and current_weight > target:
        return('Slightly Overweight')
    elif current_weight == target:
        return('On Target')
    elif current_weight < target and current_weight >= lower:
        return('Slightly Underweight')
    elif current_weight < lower:
        return('Very Underweight')
    else:
        pass


In [23]:
current_port['Status']=current_port.apply(calculate_weights, axis=1)
current_port

,Target,Upper,Lower,Current Share,Close,MV,Current Weight,Status
FB,0.2,0.25,0.15,100,184.6700,18467.00,0.064697,Very Underweight
AMZN,0.2,0.25,0.15,100,1204.2000,120420.00,0.421875,Very Overweight
AAPL,0.2,0.25,0.15,100,168.9578,16895.78,0.059192,Very Underweight
NFLX,0.2,0.25,0.15,100,205.0500,20505.00,0.071837,Very Underweight
GOOGL,0.2,0.25,0.15,100,1091.5200,109152.00,0.382399,Very Overweight
Cash,0.0,0.00,0.00,0,1.0000,0.00,0.000000,On Target


In [24]:
# Add target market value
current_port['Target MV']=current_port['Target'] * port_val_rebalance
current_port

,Target,Upper,Lower,Current Share,Close,MV,Current Weight,Status,Target MV
FB,0.2,0.25,0.15,100,184.6700,18467.00,0.064697,Very Underweight,57087.956
AMZN,0.2,0.25,0.15,100,1204.2000,120420.00,0.421875,Very Overweight,57087.956
AAPL,0.2,0.25,0.15,100,168.9578,16895.78,0.059192,Very Underweight,57087.956
NFLX,0.2,0.25,0.15,100,205.0500,20505.00,0.071837,Very Underweight,57087.956
GOOGL,0.2,0.25,0.15,100,1091.5200,109152.00,0.382399,Very Overweight,57087.956
Cash,0.0,0.00,0.00,0,1.0000,0.00,0.000000,On Target,0.000


In [25]:
# Now solve for new Share Amounts
# only rebalance back to target if Very Overweight or Very Underweight

solve_weights = []

# includes cash
number_of_positions=6

cash = 0

for r in range(0, number_of_positions):
    #Share count
    allocation = current_port.iloc[r,3]
    
    # Share price
    share_price = current_port.iloc[r,4]
    
    # Current Equity Value
    current_mv = current_port.iloc[r,5]
    
    # Target Equity Value
    target_mv = current_port.iloc[r,8]
    
    #weighting status
    status = current_port.iloc[r,7]
    
    if (status == "Very Overweight") | (status == "Very Underweight"):
        difference = target_mv - current_mv
        shares = difference // share_price
        solve_weights.append(allocation + shares)
        cash += difference % share_price
    
    elif (status == 'Slightly Overweight') | (status == 'On Target') | (status == 'Slightly Underweight'):
        solve_weights.append(allocation)
    
    
solve_weights
    

[309.0, 47.0, 337.0, 278.0, 52.0, 0]

In [45]:
cash

In [27]:
solve_weights = pd.DataFrame({rebalance_day: tickers, 
                  'New Share Amount': solve_weights}).set_index(rebalance_day)
        
solve_weights

,New Share Amount
2018-01-03,
FB,309.0
AMZN,47.0
AAPL,337.0
NFLX,278.0
GOOGL,52.0
Cash,0.0


In [28]:
# Add new Share amount to column
current_port = pd.concat([current_port, solve_weights], axis=1)
current_port.index.rename(rebalance_day)
current_port

,Target,Upper,Lower,Current Share,Close,MV,Current Weight,Status,Target MV,New Share Amount
2018-01-03,,,,,,,,,,
FB,0.2,0.25,0.15,100,184.6700,18467.00,0.064697,Very Underweight,57087.956,309.0
AMZN,0.2,0.25,0.15,100,1204.2000,120420.00,0.421875,Very Overweight,57087.956,47.0
AAPL,0.2,0.25,0.15,100,168.9578,16895.78,0.059192,Very Underweight,57087.956,337.0
NFLX,0.2,0.25,0.15,100,205.0500,20505.00,0.071837,Very Underweight,57087.956,278.0
GOOGL,0.2,0.25,0.15,100,1091.5200,109152.00,0.382399,Very Overweight,57087.956,52.0
Cash,0.0,0.00,0.00,0,1.0000,0.00,0.000000,On Target,0.000,0.0


In [30]:
# Return tear sheet of buy and sell orders
def calculate_shares(row):
    status = row[7]
    current_share = row[3]
    new_share = row[9]
    
    if (status == "Very Overweight") | (status == "Very Underweight"):
        return new_share-current_share 
    
    elif (status == 'Slightly Overweight') | (status == 'On Target') | (status == 'Slightly Underweight'):
        return current_share

In [34]:
# Add the number of shares to buy or sell
current_port['# Buy/Sell'] = current_port.apply(calculate_shares, axis=1)
current_port

,Target,Upper,Lower,Current Share,Close,MV,Current Weight,Status,Target MV,New Share Amount,# Buy/Sell
2018-01-03,,,,,,,,,,,
FB,0.2,0.25,0.15,100,184.6700,18467.00,0.064697,Very Underweight,57087.956,309.0,209.0
AMZN,0.2,0.25,0.15,100,1204.2000,120420.00,0.421875,Very Overweight,57087.956,47.0,-53.0
AAPL,0.2,0.25,0.15,100,168.9578,16895.78,0.059192,Very Underweight,57087.956,337.0,237.0
NFLX,0.2,0.25,0.15,100,205.0500,20505.00,0.071837,Very Underweight,57087.956,278.0,178.0
GOOGL,0.2,0.25,0.15,100,1091.5200,109152.00,0.382399,Very Overweight,57087.956,52.0,-48.0
Cash,0.0,0.00,0.00,0,1.0000,0.00,0.000000,On Target,0.000,0.0,0.0


In [35]:
# Add the action for clarity
def determine_action(row):
    status = row[7]

    if (status == "Very Overweight"):
        return 'Sell'
    
    elif (status == "Very Underweight"):
        return 'Buy' 
    
    elif (status == 'Slightly Overweight') | (status == 'On Target') | (status == 'Slightly Underweight'):
        return 'Do Nothing'

In [39]:
current_port['Action']=current_port.apply(determine_action, axis=1)
current_port

,Target,Upper,Lower,Current Share,Close,MV,Current Weight,Status,Target MV,New Share Amount,# Buy/Sell,Action
2018-01-03,,,,,,,,,,,,
FB,0.2,0.25,0.15,100,184.6700,18467.00,0.064697,Very Underweight,57087.956,309.0,209.0,Buy
AMZN,0.2,0.25,0.15,100,1204.2000,120420.00,0.421875,Very Overweight,57087.956,47.0,-53.0,Sell
AAPL,0.2,0.25,0.15,100,168.9578,16895.78,0.059192,Very Underweight,57087.956,337.0,237.0,Buy
NFLX,0.2,0.25,0.15,100,205.0500,20505.00,0.071837,Very Underweight,57087.956,278.0,178.0,Buy
GOOGL,0.2,0.25,0.15,100,1091.5200,109152.00,0.382399,Very Overweight,57087.956,52.0,-48.0,Sell
Cash,0.0,0.00,0.00,0,1.0000,0.00,0.000000,On Target,0.000,0.0,0.0,Do Nothing


In [40]:
print ("---------Number of Shares to Buy/Sell------------")
print(pd.concat([current_port['Action'], current_port['# Buy/Sell']], axis=1))

---------Number of Shares to Buy/Sell------------
                Action  # Buy/Sell
2018-01-03                        
FB                 Buy       209.0
AMZN              Sell       -53.0
AAPL               Buy       237.0
NFLX               Buy       178.0
GOOGL             Sell       -48.0
Cash        Do Nothing         0.0


## Rebalance the holdings and create excel file for trade orders

In [77]:
rebalanced_port = current_port.copy()
rebalanced_port['Current Share'] = rebalanced_port['New Share Amount']
rebalanced_port.loc['Cash', 'Current Share'] = cash
rebalanced_port['MV'] = rebalanced_port['Current Share'] * rebalanced_port['Close']
rebalanced_port.drop(labels=['New Share Amount', '# Buy/Sell', 'Target MV', 'Action'], axis=1, inplace=True)

In [78]:
rebalanced_port_value = rebalanced_port['MV'].sum()
rebalanced_port_value

285439.77999999997

In [79]:
rebalanced_port['Current Weight'] = rebalanced_port['MV'] / rebalanced_port_value
rebalanced_port['Status'] = rebalanced_port.apply(calculate_weights, axis=1)
rebalanced_port

,Target,Upper,Lower,Current Share,Close,MV,Current Weight,Status
2018-01-03,,,,,,,,
FB,0.2,0.25,0.15,309.0000,184.6700,57063.0300,0.199913,Slightly Underweight
AMZN,0.2,0.25,0.15,47.0000,1204.2000,56597.4000,0.198281,Slightly Underweight
AAPL,0.2,0.25,0.15,337.0000,168.9578,56938.7786,0.199477,Slightly Underweight
NFLX,0.2,0.25,0.15,278.0000,205.0500,57003.9000,0.199706,Slightly Underweight
GOOGL,0.2,0.25,0.15,52.0000,1091.5200,56759.0400,0.198848,Slightly Underweight
Cash,0.0,0.00,0.00,1077.6314,1.0000,1077.6314,0.003775,Very Overweight


In [80]:
rebalanced_port.to_csv(rebalance_day,index=True)